# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Tismark Boham

**ID**: etb62

> **Due Date**
>
> Wednesday, 10/16/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/4750coding/Labs/lab-3-TismarkBoham`
   Installed CommonSubexpressions ─ v0.3.0
   Installed StaticArraysCore ───── v1.4.3
   Installed HiGHS_jll ──────────── v1.7.2+0
   Installed BenchmarkTools ─────── v1.5.0
   Installed MutableArithmetics ─── v1.4.6
   Installed CodecBzip2 ─────────── v0.8.3
   Installed DiffResults ────────── v1.1.0
   Installed DiffRules ──────────── v1.15.1
   Installed HiGHS ──────────────── v1.9.2
   Installed ForwardDiff ────────── v0.10.36
   Installed JuMP ───────────────── v1.22.2
   Installed MathOptInterface ───── v1.31.0
Precompiling project...
  ✓ StaticArraysCore
  ✓ CommonSubexpressions
  ✓ libvorbis_jll
  ✓ HiGHS_jll
  ✓ CodecBzip2
  ✓ GLFW_jll
  ✓ DiffRules
  ✓ Qt6ShaderTools_jll
  ✓ DiffResults
  ✓ BenchmarkTools
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ GR_jll
  ✓ Qt6Declarative_jll
  ✓ Qt6Wayland_jll
  ✓ MutableArithmetics
  ✓ ForwardDiff
  ✓ GR
  ✓ Plots
  ✓ Plots → UnitfulExt
  ✓ MathOptInterface
  ✓ HiGHS
  ✓ JuMP
  23 dependenc

In [2]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

`JuMP` requires the loading of a solver. \[Each supported solver works
for certain classes of problems, and some are open source while others
require a commercial license\]. We will use the `HiGHS` solver, which is
open source and works for linear, mixed integer linear, and quadratic
programs.

In this lab we will walk through the steps involved in coding a linear
program in HiGHS, solving it, and querying the solution.

## Exercise (3 points)

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
$\$0.12$ per bf to purchase and fir costs $\$0.08$ per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board, which yields no
revenue for the mill.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

First, we need to identify our decision variables. While there are
several options, we will use $G_i$, the amount of each grade the mill
produces (in \$/1000 bf).

Using these decision variables, formulate a linear program to maximize
the profit of the mill subject to the supply constraints on spruce and
fir.

> **JuMP Syntax**
>
> The core pieces of setting up a `JuMP` model involve specifying the
> model and adding variables, the objective, and constraints. At the
> most simple level, this syntax looks like this:
>
> ``` julia
> m = Model(HiGHS.Optimizer)
> @variable(m, lb <= x <= ub) # if you do not have upper or lower bounds, you can drop those accordingly
> @variable(m, lb <= y <= ub)
> @objective(m, Max, 100x + 250y) # replace Max with Min depending on the problem
> @constraint(m, label, 6x + 12y <= 80) # replace "label" with some meaningful string you would like to use later to query shadow prices, or drop it
> ```
>
> You can add more constraints or more variables as needed.

> **Using Array Syntax**
>
> You can set up multiple variables or constraints at once using array
> syntax. For example, the following are equivalent:
>
> ``` julia
> @variable(m, G1 >= 0)
> @variable(m, G2 >= 0)
> @variable(m, G3 >= 0)
> ```
>
> and
>
> ``` julia
> @variable(m, G[1:3] >= 0)
> ```
>
> You can also set up multiple constraints using arrays of coefficients
> and/or bounds. For example:
>
> ``` julia
> I = 1:3
> d = [0; 3; 5]
> @constraint(m, demand[i in I], G[i] >= d[i])
> ```

`JuMP` is finicky about changing objects and constraints, so I recommend
setting up all of the model syntax in one notebook cell, which is what
we will do here.

First, we have to use JuMP format to set our objectives and constraints. 

OBJECTIVE - Maximize Profit

Profit = Revenue - Cost\
Considering all grades and simplifying the equation, the profit equation is: \
Profit = $220G_1 + 240G_2 + 360G_3$

CONSTRAINTS - amount of wood available

Spruce Constraint: $320,000 \geq G_1*500 + G_2*1000 + G_3*1500$\
Fir Constraint: $720,000 \geq G_1*1500 + G_2*2000 + G_3*2000$

G1 refers to Grade 1, which is a product of ours with specific requirements in its production. These requirements are incorporated into our Profit equation (costs to make the grade).

G2 and G3 refer to Grade 2 and Grade 3.

In [11]:
forest_model = Model(HiGHS.Optimizer) # initialize model object
@variable(forest_model, G[1:3] >= 0) # non-negativity constraints
# uncomment the following lines and add the objective and constraints as needed for the model
@objective(forest_model, Max, 220G[1]+240G[2]+360G[3])
 @constraint(forest_model,spruce, 320000 >= G[1]*500 + G[2]*1000 + G[3]*1500)
 @constraint(forest_model,fir, 720000 >= G[1]*1500 + G[2]*2000 + G[3]*2000)
print(forest_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 220 G[1] + 240 G[2] + 360 G[3]
Subject to
 spruce : -500 G[1] - 1000 G[2] - 1500 G[3] ≥ -320000
 fir : -1500 G[1] - 2000 G[2] - 2000 G[3] ≥ -720000
 G[1] ≥ 0
 G[2] ≥ 0
 G[3] ≥ 0


Next, to optimize, use the `optimize!()` function:

In [12]:
optimize!(forest_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e+02, 2e+03]
  Cost   [2e+02, 4e+02]
  Bound  [0e+00, 0e+00]
  RHS    [3e+05, 7e+05]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.2499928695e-01 Ph1: 2(6.5918); Du: 3(0.624999) 0s
          2     1.1200000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


You should get confirmation that a solution was found; if one was not,
there’s a chance something was wrong with your model formulation.

To find the values of the decision variables, use `value()` (which can
be broadcasted over variable arrays):

In [13]:
@show value.(G);

value.(G) = [352.0, 0.0, 96.0]


This shows us how much of each grade we should produce and sell. We should sell 352,000bf of Grade 1 and 96,000bf of Grade 3 and 0bf of Grade 2 to maximize profit.

Similarly, `objective_value()` finds the optimal value of the objective:

In [6]:
@show objective_value(forest_model);

objective_value(forest_model) = 112000.0


This means that our maximum profit with the given situation is $112,000

Finally, we can find the dual values of the constraints with
`shadow_price()`. Do this for the constraints in your model using the
block below.

In [14]:
 @show shadow_price(spruce);
 @show shadow_price(fir);


shadow_price(spruce) = 0.08000000000000002
shadow_price(fir) = 0.12


The shadow price tells us whether a constraint is binding or not. If the shadow price is 0, it means that the constraint is not binding; esentially, if we loosen the constraint our objective will not be effected. If the shadow price is non-zero, it means that one unit change to our constraint's boundary will effect the objective by that non-zero amount.

`JuMP` also lets you evaluate other expressions that you might be
interested in based on the solutions. For example, you can use the
following block to calculate the total amount of plywood the mill would
produce under the optimal solution:

In [10]:
@expression(forest_model, total_plywood, sum(G))
@show value.(total_plywood);

value.(total_plywood) = 448.0


This is how many total pieces of plywood is sold. This is a function that we made ourself, by summing all of the grades that we need to sell in our optimized model.

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

I worked with Cella Schnabel